In [1]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

In [2]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
from visualize_pyphi import *
from visualize_pyphi import utils
from visualize_pyphi import compute
import pickle as pkl
from tqdm.auto import tqdm
import numpy as np


Welcome to PyPhi!

If you use PyPhi in your research, please cite the paper:

  Mayner WGP, Marshall W, Albantakis L, Findlay G, Marchman R, Tononi G.
  (2018). PyPhi: A toolbox for integrated information theory.
  PLOS Computational Biology 14(7): e1006343.
  https://doi.org/10.1371/journal.pcbi.1006343

Documentation is available online (or with the built-in `help()` function):
  https://pyphi.readthedocs.io

To report issues, please use the issue tracker on the GitHub repository:
  https://github.com/wmayner/pyphi

For general discussion, you are welcome to join the pyphi-users group:
  https://groups.google.com/forum/#!forum/pyphi-users

To suppress this message, either:
  - Set `WELCOME_OFF: true` in your `pyphi_config.yml` file, or
  - Set the environment variable PYPHI_WELCOME_OFF to any value in your shell:
        export PYPHI_WELCOME_OFF='yes'



In [162]:
# ch3 system creation
net_name = "ch3_v5"

th = 1 / 4
exp = 5
mu = 1
si = 0.3
ll = 1
kk = 15
x0 = 0.7

noise = 0.00
rule = (weights>-1)

s = 0.8
l = 0.25
f = 0.0
q = 0.6
j = 1 - q
i = j - 0.05
b = 0.01

weights = np.array(
    [
        [s, l, f, i, 0, 0],  # A
        [l, s, l, j, b, 0],  # B
        [f, l, s, i, 0, 0],  # C
        [b, b, b, q, 0, i],  # D
        [0, i, 0, 0, 0, 0],  # I
        [0, 0, 0, 0, 0, 0],  # O
        # A, B, C, D, I, O
    ]
)

weights += rule * np.random.random((6,6))*noise
weights = np.abs(weights)

node_labels = ["A", "B", "C", "D", "I", "O"]
mech_func = [f for f in "sssgll"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    mu=mu,
    si=si,
    l=ll,
    k=kk,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 1, 0, 1, 1, 1)
subsystem = pyphi.Subsystem(network, state, nodes=(0,))

  0%|          | 0/64 [00:00<?, ?it/s]

Network saved to: ch3_v5


In [163]:
ces = utils.flatten([[d.cause,d.effect] for d in pyphi.compute.ces(subsystem,parallel=True)])
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))

there are 1 possible compositional states


In [164]:
max_ces = max(
    [
        compute.filter_ces(subsystem, ces, compositional_state, max_relations_k=3)
        for compositional_state in tqdm(all_compositional_states)
    ],
    key=lambda c: c["big phi"],
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [135]:

fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/max_ces',surface_opacity=0.7,)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [165]:
max_ces['big phi']

1.361076302171824e+00

In [166]:
max_ces['MIP']

(((), ()), 'disintegration')

In [167]:
utils.sepces2df(max_ces['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.680538
1,A,EFFECT,A,0,0.784442


In [168]:
ces

[Maximally-irreducible cause
   φ = 0.680538
   Mechanism: [A]
   Purview = [A]
   State = None
   Direction: CAUSE,
 Maximally-irreducible effect
   φ = 0.784442
   Mechanism: [A]
   Purview = [A]
   State = None
   Direction: EFFECT]

In [50]:
utils.sepces2df(max_ces['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.532197
1,A,EFFECT,A,0,0.666480
2,B,CAUSE,B,1,0.440708
3,B,EFFECT,B,1,0.557322
4,C,CAUSE,C,0,0.537943
5,C,EFFECT,C,0,0.671042
6,AB,CAUSE,AB,10,0.171851
7,AB,EFFECT,ABD,010,0.040551
8,AC,CAUSE,ABC,100,0.080356
9,AC,EFFECT,ABCD,0000,0.150566


In [72]:
utils.sepces2df(ces,subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.486034
1,A,EFFECT,A,0,0.627340
2,B,CAUSE,B,0,0.601248
3,B,EFFECT,B,0,0.703992
4,C,CAUSE,C,0,0.486034
5,C,EFFECT,C,0,0.627340
6,D,CAUSE,BD,00,0.311921
7,D,EFFECT,D,0,0.177053
8,AB,CAUSE,AB,00,0.075913
9,AB,EFFECT,ABD,000,0.185003


In [76]:
CES = [m for f in filtered for m in f if m['big phi']>8.8][0]
utils.sepces2df(CES['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.512357
1,A,EFFECT,A,0,0.649867
2,B,CAUSE,B,1,0.452130
3,B,EFFECT,B,1,0.567426
4,C,CAUSE,C,0,0.514622
5,C,EFFECT,C,0,0.652147
6,AB,CAUSE,AB,10,0.180604
7,AB,EFFECT,ABD,010,0.038629
8,BC,CAUSE,BC,01,0.175186
9,BC,EFFECT,BCD,100,0.038340


In [77]:
CES['MIP']

(((0,), (1, 2, 3)), (1, 2, 3), (0,), EFFECT)

In [69]:
good_weights = weights
good_weights

array([[0.80429078, 0.25524274, 0.00307273, 0.3515403 , 0.00492981,
        0.00996336],
       [0.25475904, 0.8015509 , 0.25453847, 0.40018269, 0.01702802,
        0.0043879 ],
       [0.00997719, 0.25221898, 0.80581981, 0.35990666, 0.00266345,
        0.00728655],
       [0.01578615, 0.01329649, 0.0181051 , 0.60958749, 0.00620138,
        0.35398605],
       [0.00974372, 0.35204232, 0.00702253, 0.00453444, 0.00349476,
        0.00514265],
       [0.00515462, 0.00230007, 0.00979452, 0.00469913, 0.00642495,
        0.00897918]])

In [129]:
# ch3 larger system creation
net_name = "ch3_v5"

th = 1 / 4
exp = 5
mu = 1
si = 0.3
ll = 1
kk = 15
x0 = 0.7

noise = 0.02
rule = (weights>0)

s = 0.8
l = 0.25
f = 0.0
q = 0.6
j = 1 - q
i = j - 0.05
b = 0.01

weights = np.array(
    [
        [s, l, f, 0, i, 0],  # A
        [l, s, l, f, j, b],  # B
        [f, l, s, l, i, 0],  # C
        [0, f, l, s, i, 0],  # D
        [b, b, b, b, q, 0],  # E
        [0, i, 0, 0, 0, 0],  # I
       # A, B, C, D, E, I
    ]
)

weights += rule * np.random.random((6,6))*noise
weights = np.abs(weights)

node_labels = ["A", "B", "C", "D", "E", "I"]
mech_func = [f for f in "ssssgl"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    mu=mu,
    si=si,
    l=ll,
    k=kk,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 1, 0, 0, 1, 1)
subsystem = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3, 4))

  0%|          | 0/64 [00:00<?, ?it/s]

Network saved to: ch3_v5


In [130]:
ces = utils.flatten([[d.cause,d.effect] for d in pyphi.compute.ces(subsystem,parallel=True)])
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))



Computing concepts:   0%|                                                                        | 0/31 [00:00<?, ?it/s]

Computing concepts:   3%|██                                                              | 1/31 [00:02<01:22,  2.73s/it]

Computing concepts:  26%|████████████████▌                                               | 8/31 [00:02<00:06,  3.61it/s]

Computing concepts:  52%|████████████████████████████████▌                              | 16/31 [00:03<00:01,  7.59it/s]

Computing concepts:  77%|████████████████████████████████████████████████▊              | 24/31 [00:03<00:00, 12.92it/s]

Computing concepts:  94%|██████████████████████████████████████████████████████████▉    | 29/31 [00:04<00:00,  9.61it/s]

                                                                                                                        

there are 384 possible compositional states


In [131]:
max_ces = max(
    [
        compute.filter_ces(subsystem, ces, compositional_state, max_relations_k=3)
        for compositional_state in tqdm(all_compositional_states)
    ],
    key=lambda c: c["big phi"],
)

  0%|          | 0/384 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [120]:

fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/max_ces',surface_opacity=0.7,)

Computing edges:   0%|          | 0/33 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/32 [00:00<?, ?it/s]

In [113]:
mice = max_ces['ces'][0]

In [118]:
mice.ria.node_labels

NodeLabels(('A', 'B', 'C', 'D', 'I', 'O'))

In [109]:
max_ces['big phi']

4.375449186656582e+00

In [110]:
max_ces['MIP']

(((2,), (0, 1, 3)), (0, 1, 3), (2,), EFFECT)

In [111]:
utils.sepces2df(max_ces['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.502959
1,A,EFFECT,A,0,0.641988
2,B,CAUSE,B,1,0.460940
3,B,EFFECT,B,1,0.574265
4,C,CAUSE,C,0,0.497810
5,C,EFFECT,C,0,0.637561
6,AB,CAUSE,AB,10,0.181306
7,AB,EFFECT,ABD,010,0.039542
8,BC,CAUSE,BC,01,0.181514
9,BC,EFFECT,BCD,100,0.039336
